
# AXG Growth Solver — Linear fσ8(z)

This notebook integrates the linear growth equation for **Adaptive-(Chi-Gravity)XG** in two scenarios:
- **Test A:** baryons + DE-like only (no dust-like branch).  
- **Test B:** dust-like + DE-like (Ω_m,eff≈0.32, μ≈1 on linear scales).

It compares the resulting **fσ8(z)** curves against the compiled RSD dataset:
`f_sigma8_compilation_AXG_PaperIII.csv`.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- User parameters ----
scenario = "B"       # "A" or "B"
Omega_b = 0.05       # baryons
Omega_m_eff = 0.32   # effective matter (dust-like AXG) for Test B
Omega_XG_DE = 0.68   # DE-like branch (w ~ -1)
w_XG = -1.0          # DE-like EoS (constant here)
mu_linear = 1.0      # μ on linear scales (Test A requires ~6.4 to mimic growth)

sigma8_0 = 0.811     # normalization at z=0 (Planck 2018, adjustable)
a_init = 1e-3
a_final = 1.0
N_steps = 4000

# path to data csv (already provided alongside manuscript assets)
csv_path = "/mnt/data/f_sigma8_compilation_AXG_PaperIII.csv"
df_data = pd.read_csv(csv_path)
df_data.head()


In [ ]:

def E2(a, scenario, Omega_b, Omega_m_eff, Omega_XG_DE, w_XG):
    if scenario.upper() == "A":
        # H^2/H0^2 = Omega_b a^-3 + Omega_XG_DE a^{-3(1+w)}; with w=-1 => const
        return Omega_b * a**(-3) + Omega_XG_DE
    else:
        # Test B: H^2/H0^2 = Omega_m_eff a^-3 + Omega_XG_DE (w=-1)
        return Omega_m_eff * a**(-3) + Omega_XG_DE

def dlnH_dlnA(a, scenario, Omega_b, Omega_m_eff, Omega_XG_DE, w_XG):
    # d ln H / d ln a = 1/2 * d ln E^2 / d ln a
    # E2 = sum_i Omega_i a^{-3(1+w_i)}
    if scenario.upper() == "A":
        # components: Omega_b a^-3, DE-like const
        num = (-3)*Omega_b * a**(-3)
        den = Omega_b * a**(-3) + Omega_XG_DE
    else:
        # components: Omega_m_eff a^-3, DE-like const
        num = (-3)*Omega_m_eff * a**(-3)
        den = Omega_m_eff * a**(-3) + Omega_XG_DE
    return 0.5 * (num / den)

def Omega_m_of_a(a, scenario, Omega_b, Omega_m_eff, Omega_XG_DE):
    if scenario.upper() == "A":
        num = Omega_b * a**(-3)
        den = Omega_b * a**(-3) + Omega_XG_DE
    else:
        num = Omega_m_eff * a**(-3)
        den = Omega_m_eff * a**(-3) + Omega_XG_DE
    return num / den

def integrate_growth(scenario, mu_linear, a_init, a_final, N_steps,
                     Omega_b, Omega_m_eff, Omega_XG_DE, w_XG):
    # We solve for y1 = D, y2 = dD/d ln a
    # y1' = y2
    # y2' = (3/2) Omega_m(a) mu D - [2 + H'/H] y2
    a_vals = np.exp(np.linspace(np.log(a_init), np.log(a_final), N_steps))
    y1 = np.zeros_like(a_vals)
    y2 = np.zeros_like(a_vals)
    # initial conditions in (almost) matter domination: D ~ a => D' (wrt ln a) ~ D
    y1[0] = a_init
    y2[0] = a_init

    for i in range(len(a_vals)-1):
        a = a_vals[i]
        dlnH = dlnH_dlnA(a, scenario, Omega_b, Omega_m_eff, Omega_XG_DE, w_XG)
        Om = Omega_m_of_a(a, scenario, Omega_b, Omega_m_eff, Omega_XG_DE)
        dln_a = np.log(a_vals[i+1]) - np.log(a_vals[i])

        def rhs(y1i, y2i):
            dy1 = y2i
            dy2 = (1.5 * Om * mu_linear) * y1i - (2.0 + dlnH) * y2i
            return dy1, dy2

        # RK4 in ln a
        k1_y1, k1_y2 = rhs(y1[i], y2[i])
        k2_y1, k2_y2 = rhs(y1[i] + 0.5*dln_a*k1_y1, y2[i] + 0.5*dln_a*k1_y2)
        k3_y1, k3_y2 = rhs(y1[i] + 0.5*dln_a*k2_y1, y2[i] + 0.5*dln_a*k2_y2)
        k4_y1, k4_y2 = rhs(y1[i] + dln_a*k3_y1, y2[i] + dln_a*k3_y2)

        y1[i+1] = y1[i] + (dln_a/6.0)*(k1_y1 + 2*k2_y1 + 2*k3_y1 + k4_y1)
        y2[i+1] = y2[i] + (dln_a/6.0)*(k1_y2 + 2*k2_y2 + 2*k3_y2 + k4_y2)

    # normalize so that D(a=1)=1
    D = y1 / y1[-1]
    f = y2 / y1  # d ln D / d ln a
    return a_vals, D, f

# Integrate for current parameters
a_vals, D, f = integrate_growth(scenario, mu_linear, a_init, a_final, N_steps,
                                Omega_b, Omega_m_eff, Omega_XG_DE, w_XG)

z_vals = 1.0/a_vals - 1.0
# Interpolate theory fσ8 at data redshifts
from numpy import interp
f_sigma8_th = interp(1.0/(1.0+df_data["z"].values), a_vals, f * D * sigma8_0)

# Simple chi^2
chi2 = np.sum(((f_sigma8_th - df_data["f_sigma8"].values)/df_data["err"].values)**2)
print(f"Scenario = {scenario} | mu_linear = {mu_linear}")
print(f"Chi^2 (illustrative) = {chi2:.2f} for N={len(df_data)} points")


In [ ]:

# Plot
plt.figure(figsize=(6,4))
plt.errorbar(df_data["z"], df_data["f_sigma8"], yerr=df_data["err"], fmt='o', label="Observed fσ8")
# also show theory curve
plt.plot(z_vals, f*D*sigma8_0, '-', label=f"AXG theory ({scenario}), μ={mu_linear}")
plt.gca().invert_xaxis()  # optional: higher z on left
plt.xlabel("z")
plt.ylabel("fσ8(z)")
plt.title("AXG Growth vs Observations")
plt.legend()
plt.tight_layout()
plt.show()
